<!-- dom:TITLE: Polynomial Chaos with the Ishigami Function -->
# Sensitivity Analysis with the Ishigami Function
<!-- dom:AUTHOR: Jacob Sturdy at Department of Structural Engineering, NTNU -->
<!-- Author: -->  
This Notebook was started by **Jacob Sturdy**, Department of Structural Engineering, NTNU in Jun 27, 2017.<br>
**Miodrag Bolic** modified it by adding Sobol and Morris analysis based on the package **SALib**. 


In [102]:
# ipython magic
%matplotlib notebook
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [103]:
%matplotlib inline

# plot configuration
import matplotlib
import matplotlib.pyplot as plt
plt.style.use("ggplot")
# import seaborn as sns # sets another style
matplotlib.rcParams['lines.linewidth'] = 3
fig_width, fig_height = (7.0,5.0)

matplotlib.rcParams['figure.figsize'] = (fig_width, fig_height)

# font = {'family' : 'sans-serif',
#         'weight' : 'normal',
#         'size'   : 18.0}
# matplotlib.rc('font', **font)  # pass in the font dict as kwar

In [104]:
import chaospy as cp
import numpy as np
from numpy import linalg as LA

# Introduction
<div id="sec:introduction"></div>

The Ishigami function

<!-- Equation labels as ordinary links -->
<div id="_auto1"></div>

$$
\begin{equation}
y = \sin(z_1) + a \sin^2(z_2) + b z_3^4 \sin(z_1)
\label{_auto1} \tag{1}
\end{equation}
$$

is a commonly used benchmark function for sensitivity analysis. You will now get the chance to evaluate polynomial chaos on this function.

Each component of $\mathbf{Z}$ is distributed uniformly over the range $(-\pi, \pi)$ and $a=7$ while $b=0.1$.

These two functions implement the function in python as well as the exact solutions for its mean, variance and sensitivities.

In [105]:
def ishigami_function(sample):
    q1 = sample[0]
    q2 = sample[1]
    q3 = sample[2]
    a = 7.
    b = 0.1
    return np.sin(q1) + a*np.sin(q2)**2 + b* q3**4 * np.sin(q1)

def ishigami_analytic():
    #Analytical values
    #Total variance
    measures = {}
    a = 7.
    measures["mean"] = a/2.0
    b = 0.1
    D = a**2./8 + b*np.pi**4./5 + b**2*np.pi**8./18 + 1./2
    measures 
    measures["var"] = D
    # Conditional variances
    D1 = b*np.pi**4./5 + b**2*np.pi**8./50. + 1./2
    D2 = a**2/8.
    D3 = 0
    
    D12  = 0
    D13  = b**2. * np.pi**8 / 18 - b**2*np.pi**8./50.
    D23  = 0
    D123 = 0
    
    # Main and total sensitivity indices
    measures["sens_m"] = {}
    measures["sens_m"][0] = D1/D
    measures["sens_m"][1] = D2/D
    measures["sens_m"][2] = D3/D
   

    measures["sens_t"] = {}
    measures["sens_t"][0] = (D1 + D12 + D13 + D123)/D
    measures["sens_t"][1] = (D2 + D12 + D23 + D123)/D
    measures["sens_t"][2] = (D3 + D13 + D23 + D123)/D
    return measures

Compare the results you get for different methods of performing polynomial chaos with this function. What sampling methods work best?

In [106]:
measures=ishigami_analytic()

In [107]:
measures

{'mean': 3.5,
 'sens_m': {0: 0.31390519114781146, 1: 0.4424111447900409, 2: 0.0},
 'sens_t': {0: 0.5575888552099592,
  1: 0.4424111447900409,
  2: 0.24368366406214773},
 'var': 13.844587940719254}

In [108]:
from SALib.sample import saltelli
from SALib.analyze import sobol
from SALib.test_functions import Ishigami
import numpy as np

# Define the model inputs
problem = {
    'num_vars': 3,
    'names': ['x1', 'x2', 'x3'],
    'bounds': [[-3.14159265359, 3.14159265359],
               [-3.14159265359, 3.14159265359],
               [-3.14159265359, 3.14159265359]]
}

# Generate samples
param_values = saltelli.sample(problem, 1000)

# Run model (example)
Y = ishigami_function (param_values.T) #Ishigami.evaluate(samples_pc.T) # #

# Perform analysis
Si = sobol.analyze(problem, Y, print_to_console=True)

# Print the first-order sensitivity indices
print (Si['S1'])

Parameter S1 S1_conf ST ST_conf
x1 0.307975 0.072571 0.560137 0.091639
x2 0.447767 0.058452 0.438722 0.039226
x3 -0.004255 0.063178 0.242845 0.025462

Parameter_1 Parameter_2 S2 S2_conf
x1 x2 0.012205 0.092464
x1 x3 0.251526 0.112856
x2 x3 -0.009954 0.066687
[ 0.30797549  0.44776661 -0.00425452]


In [109]:
from SALib.sample.morris import sample 
from SALib.analyze.morris import analyze
X = sample(problem, 1000, num_levels=4, grid_jump=2)
Y = ishigami_function(X.T)
Si_m = analyze(problem, X, Y, conf_level=0.95,
                     print_to_console=True, num_levels=4, grid_jump=2)
print (Si_m)

Parameter                         Mu_Star         Mu    Mu_Star_Conf      Sigma
x1                                  7.991      7.991           0.403      6.245
x2                                  7.875      0.236           0.000      7.875
x3                                  6.411     -0.037           0.385      8.956
{'mu_star_conf': [0.40281012695872359, 0.0, 0.38506856162742298], 'names': ['x1', 'x2', 'x3'], 'mu_star': array([ 7.99149873,  7.875     ,  6.4112648 ]), 'sigma': array([ 6.24530481,  7.87539413,  8.95567817]), 'mu': array([ 7.99149873,  0.23625   , -0.03749278])}
